In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import sys
sys.path.insert(0,'/content/drive/MyDrive/ColabNotebooks/')

In [3]:
%pip install torchmetrics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 763.4/763.4 kB 6.8 MB/s eta 0:00:00


In [4]:
import torch
from torchmetrics.classification import BinaryF1Score, BinaryAccuracy, BinarySpecificity, BinaryPrecision, BinaryRecall
from model.smooth_cross_entropy import smooth_crossentropy
from coronary import Coronary
from utility.log import Log
from utility.initialize import initialize
from utility.step_lr import StepLR
from optimizers_impl.sam.sam import SAM
from utility.bypass_bn import enable_running_stats, disable_running_stats
from optimizers_impl.gsam import GSAM, CosineScheduler, ProportionScheduler

In [5]:
def train(model, device, train_loader, optimizer, log, scheduler):
    model.train()
    log.train(len_dataset=len(dataset.train))

    for batch in train_loader:
        inputs, targets = (b.to(device) for b in batch)
        inputs = inputs.permute(0, 3, 2, 1)

        def loss_fn(predictions, targets):
            return smooth_crossentropy(predictions.logits, targets, smoothing=0.0001).mean()

        optimizer.set_closure(loss_fn, inputs, targets)
        predictions, loss = optimizer.step()

        with torch.no_grad():
            correct = torch.argmax(predictions.logits, 1) == targets
            log(model, loss.cpu(), correct.cpu(), scheduler.lr())
            scheduler.step()
            optimizer.update_rho_t()


In [6]:
def test(model, device, test_loader, log):
    model.eval()
    log.eval(len_dataset=len(test_loader))

    with torch.no_grad():
        for batch in test_loader:
            inputs, targets = (b.to(device) for b in batch)
            inputs = inputs.permute(0, 3, 2, 1)

            predictions = model(inputs)
            loss = smooth_crossentropy(predictions, targets, smoothing=0.0001)
            correct = torch.argmax(predictions, 1) == targets
            log(model, loss.cpu(), correct.cpu())

In [7]:
if __name__ == "__main__":

    num_epochs = 50
    # model_name_path = '/content/drive/MyDrive/ColabNotebooks/results_logs/'
    model_name = 'inceptionv3-gsam'
    initialize(seed=111)
    log = Log(log_each=1, log_name=model_name)

    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

    model = torch.hub.load('pytorch/vision:v0.10.0', 'inception_v3', pretrained=True).to(device)
    model.num_classes = 2
    model.eval()

    base_optimizer = torch.optim.SGD(model.parameters(), lr=0.003, momentum=0.03, weight_decay=0.3)
    # base_optimizer = torch.optim.Adagrad(model.parameters(), lr=0.005, weight_decay=0.7)

    dataset = Coronary(batch_size=100, threads=2, img_size_definition=299)

    scheduler = CosineScheduler(T_max=num_epochs * len(dataset.train), max_value=0.002, min_value=0.0, optimizer=base_optimizer)
    rho_scheduler = ProportionScheduler(pytorch_lr_scheduler=scheduler, max_lr=0.02, min_lr=0.0, max_value=1.0, min_value=1.0)
    optimizer = GSAM(params=model.parameters(), base_optimizer=base_optimizer, model=model, gsam_alpha=0.1, rho_scheduler=rho_scheduler, adaptive=True)


    for epoch in range(num_epochs):
        train(model, device, dataset.train, optimizer, log, scheduler)
        test(model, device, dataset.valid, log)

    log.flush()
# ┃          49  ┃    465.4184  │     91.56 %  ┃   9.748e-08  │       00:04  ┃    465.2818  │     79.67 %  ┃


Downloading: "https://github.com/pytorch/vision/zipball/v0.10.0" to /root/.cache/torch/hub/v0.10.0.zip
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=Inception_V3_Weights.IMAGENET1K_V1`. You can also use `weights=Inception_V3_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/inception_v3_google-0cc3c7bd.pth" to /root/.cache/torch/hub/checkpoints/inception_v3_google-0cc3c7bd.pth
100%|██████████| 104M/104M [00:01<00:00, 75.9MB/s] 


100
300
880
┏━━━━━━━━━━━━━━┳━━━━━━━╸T╺╸R╺╸A╺╸I╺╸N╺━━━━━━━┳━━━━━━━╸S╺╸T╺╸A╺╸T╺╸S╺━━━━━━━┳━━━━━━━╸V╺╸A╺╸L╺╸I╺╸D╺━━━━━━━┓
┃              ┃              ╷              ┃              ╷              ┃              ╷              ┃
┃       epoch  ┃        loss  │    accuracy  ┃        l.r.  │     elapsed  ┃        loss  │    accuracy  ┃
┠──────────────╂──────────────┼──────────────╂──────────────┼──────────────╂──────────────┼──────────────┨


/content/drive/MyDrive/ColabNotebooks/coronary.py:86: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:245.)
  train_tensor_x = torch.Tensor(train_xs)


┃           0  ┃      6.4553  │      0.89 %  ┃   1.999e-03  │       01:06  ┃      6.5302  │      0.00 %  ┃
┃           1  ┃      4.7239  │     12.56 %  ┃   1.994e-03  │       00:23  ┃      5.1611  │      0.33 %  ┃
┃           2  ┃      3.0569  │     50.00 %  ┃   1.985e-03  │       00:23  ┃      3.8315  │     33.00 %  ┃
┃           3  ┃      1.9517  │     73.56 %  ┃   1.972e-03  │       00:24  ┃      2.1912  │     77.67 %  ┃
┃           4  ┃      1.2459  │     79.56 %  ┃   1.955e-03  │       00:24  ┃      1.3432  │     74.00 %  ┃
┃           5  ┃      0.8884  │     81.33 %  ┃   1.935e-03  │       00:24  ┃      1.3233  │     71.67 %  ┃
┃           6  ┃      0.7140  │     82.22 %  ┃   1.911e-03  │       00:24  ┃      0.8674  │     76.67 %  ┃
┃           7  ┃      0.6189  │     84.89 %  ┃   1.883e-03  │       00:24  ┃      0.7785  │     79.33 %  ┃
┃           8  ┃      0.5986  │     84.11 %  ┃   1.852e-03  │       00:24  ┃      0.7833  │     78.33 %  ┃
┃           9  ┃      0.5436  │     8

In [8]:
metricF1 = BinaryF1Score().to(device)
metricAcc = BinaryAccuracy().to(device)
metricSpecificity = BinarySpecificity().to(device)
metricPrecision = BinaryPrecision().to(device)
metricRecall = BinaryRecall().to(device)

In [9]:
inputs = dataset.test_x.permute(0, 3, 2, 1).to(device)
predictions = model(inputs)
f1 = metricF1(torch.argmax(predictions, 1), dataset.test_y.to(device))
acc = metricAcc(torch.argmax(predictions, 1), dataset.test_y.to(device))
specificity = metricSpecificity(torch.argmax(predictions, 1), dataset.test_y.to(device))
precision = metricPrecision(torch.argmax(predictions, 1), dataset.test_y.to(device))
recall = metricRecall(torch.argmax(predictions, 1), dataset.test_y.to(device))

with open(model_name+'.txt', 'a') as file:
                file.write(f"\n \n F1: {f1}\n Accuracy: {acc}\n Specificity:{specificity}\n Precision: {precision}\n Recall: {recall}")

print(f1)
print(acc)
print(precision)
print(specificity)
print(recall)


tensor(0.9804, device='cuda:0')
tensor(0.9800, device='cuda:0')
tensor(0.9615, device='cuda:0')
tensor(0.9600, device='cuda:0')
tensor(1., device='cuda:0')


In [10]:
# if __name__ == "__main__":

#     num_epochs = 50
#     model_name_path = '/content/drive/MyDrive/ColabNotebooks/results_logs/'
#     model_name = 'siema'
#     initialize(seed=11)
#     log = Log(log_each=1, log_name=model_name)

#     device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

#     model = torch.hub.load('pytorch/vision:v0.10.0', 'inception_v3', pretrained=True).to(device)
#     model.num_classes = 2
#     model.eval()

#     # base_optimizer = torch.optim.SGD(model.parameters(), lr=0.0005, momentum=0.1, weight_decay=1e-4)
#     base_optimizer = torch.optim.Adagrad(model.parameters(), lr=0.003, weight_decay=0.0)

#     dataset = Coronary(batch_size=100, threads=2, img_size_definition=299)

#     scheduler = CosineScheduler(T_max=num_epochs * len(dataset.train), max_value=0.003, min_value=0.0, optimizer=base_optimizer)
#     rho_scheduler = ProportionScheduler(pytorch_lr_scheduler=scheduler, max_lr=0.02, min_lr=0.0, max_value=2.0, min_value=2.0)
#     optimizer = GSAM(params=model.parameters(), base_optimizer=base_optimizer, model=model, gsam_alpha=0.01, rho_scheduler=rho_scheduler, adaptive=True)


#     for epoch in range(num_epochs):
#         train(model, device, dataset.train, optimizer, log, scheduler)
#         test(model, device, dataset.valid, log)

#     log.flush()
